In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.optim as optim



In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [54]:
train_data = datasets.CIFAR10('data',train = True, transform=transform,  download=True)
test_data = datasets.CIFAR10('data',train = False, transform=transform, download=True )



Files already downloaded and verified
Files already downloaded and verified


In [0]:
batch = 20

train_size = len(train_data)
# print(train_size)
valid_size = 0.2

indices = list(range(train_size))
valid_size = int(np.floor(0.2*train_size))
# print(valid_size)

train_idx = indices[valid_size:]
valid_idx = indices[:valid_size]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = torch.utils.data.DataLoader(train_data,sampler = train_sampler, batch_size=batch,shuffle = False)
valid_loader = torch.utils.data.DataLoader(train_data,sampler = valid_sampler, batch_size=batch,shuffle = False)
test_loader =  torch.utils.data.DataLoader(test_data, batch_size=batch,shuffle = True)

classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']


In [56]:
for data, labels in train_loader:
  print(data.shape)
  print(labels)
  break


torch.Size([20, 3, 32, 32])
tensor([7, 6, 9, 5, 6, 9, 9, 0, 2, 3, 6, 8, 2, 4, 3, 4, 6, 8, 8, 5])


In [57]:
class CNN(nn.Module):

  def __init__(self):
    super(CNN,self).__init__()
    self.conv1 = nn.Conv2d(3,32,(3,3),1,1)
    self.conv2 = nn.Conv2d(32,64,(3,3),1,1)
    self.conv3 = nn.Conv2d(64,128,(3,3),1,1)
    self.conv4 = nn.Conv2d(128,64,(3,3),1,1) 

    self.fc1 = nn.Linear(256,128)
    self.fc2 = nn.Linear(128,64)
    self.fc3 = nn.Linear(64,10)

    self.maxpool = nn.MaxPool2d((2,2))


  def forward(self,x):
    x=F.relu(self.conv1(x))
    x=self.maxpool(x)
    # print(x.shape)
    x=F.relu(self.conv2(x))
    x=self.maxpool(x)
    # print(x.shape)
    x=F.relu(self.conv3(x))
    x=self.maxpool(x)
    # print(x.shape)
    x=F.relu(self.conv4(x))
    x=self.maxpool(x)
    # print(x.shape)
    x=x.view(-1,x.shape[1]*x.shape[2]*x.shape[3])
    # print(x.shape)
    x=F.relu(self.fc1(x))
    x=F.relu(self.fc2(x))
    x=self.fc3(x)
    return(x)


cnn = CNN()
cnn.to(device)


CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
  (maxpool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
)

In [0]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn.parameters(), lr=0.01,momentum=0.9)


In [59]:
epoch = 30
min_val_loss=np.Inf
for i in range(epoch):
  correct_pred = 0
  correct_pred_v=0
  running_loss_v=0
  running_loss=0
  
  
  for data,label in train_loader:
    
    optimizer.zero_grad()
    # print(data.shape)
    p = cnn.forward(data.to(device))
    prob = F.softmax(p,1)
    # print(prob)
    _,pred_class = torch.max(prob,1)
    correct_pred += torch.sum(pred_class == label.to(device)).item()
    # print(correct_pred)  
    loss=criterion(p,label.to(device))
    # print(data.shape[0])
    loss.backward()
    running_loss+=loss.item()*data.shape[0]
    optimizer.step()
  
  # Validation:

  for data_v,label_v in valid_loader:
    data_v, label_v = data_v.to(device), label_v.to(device)
    p_v = cnn.forward(data_v)
    prob_v = F.softmax(p_v,1)
    _,pred_class_v = torch.max(prob_v,1)
    correct_pred_v += torch.sum(pred_class_v == label_v).item()
    # print(correct_pred_v)
    loss_v=criterion(p_v,label_v)
    running_loss_v+=loss_v.item()*data_v.shape[0]

  print("Training Dataset: ")    
  print("Epoch: {}  Loss: {}  Accuracy: {}".format(i,running_loss/len(train_loader.sampler),correct_pred/len(train_loader.sampler)))
  print()
  print('*'*50)
  print()
  print('Validation: ')
  print('Loss: {}  Accuracy: {}'.format(running_loss_v/len(valid_loader.sampler),correct_pred_v/len(valid_loader.sampler)))
  print()
  if (running_loss_v/len(valid_loader.sampler) < min_val_loss):
    print('Saving the model')
    torch.save(cnn.state_dict(),'checkpoint.pt')
    min_val_loss = running_loss_v/len(valid_loader.sampler)
    print('New minimum validation loss:',min_val_loss)
  print()
  print()

    





Training Dataset: 
Epoch: 0  Loss: 1.8192537057697773  Accuracy: 0.31315

**************************************************

Validation: 
Loss: 1.5205116658210753  Accuracy: 0.454

Saving the model
New minimum validation loss: 1.5205116658210753


Training Dataset: 
Epoch: 1  Loss: 1.2478242857456208  Accuracy: 0.54495

**************************************************

Validation: 
Loss: 1.1044011603593826  Accuracy: 0.6034

Saving the model
New minimum validation loss: 1.1044011603593826


Training Dataset: 
Epoch: 2  Loss: 1.0006090006232262  Accuracy: 0.6477

**************************************************

Validation: 
Loss: 0.89627119743824  Accuracy: 0.6866

Saving the model
New minimum validation loss: 0.89627119743824


Training Dataset: 
Epoch: 3  Loss: 0.868380557090044  Accuracy: 0.6952

**************************************************

Validation: 
Loss: 0.8462098114490509  Accuracy: 0.7045

Saving the model
New minimum validation loss: 0.8462098114490509


Training

In [66]:
cnn.load_state_dict(torch.load('checkpoint.pt'))

# Testing 

cnn.eval()
with torch.no_grad():
  correct_pred = 0
  for data,label in test_loader:
    data,label = data.to(device),label.to(device)
    output = cnn.forward(data)
    prob = F.softmax(output,1)
    _,pred_class = torch.max(prob,1)
    correct_pred += torch.sum(pred_class == label).item()

  print('Accuracy on test Data:',correct_pred/len(test_loader.sampler))


Accuracy on test Data: 0.7305
